Import Libraries

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.datasets import mnist
from keras import backend as k

Variables:
batch: the process of splitting the training dataset in n batches (mini-batches),
classes: number of classifications (labels) of the data,
epochs: variations, one epoch is one forward pass + one backward pass on training

In [ ]:
batch_size = 128
num_classes = 10
epochs = 4

Assign training and test data

In [ ]:
img_rows, img_cols = 28,28
(x_train,y_train),(x_test,y_test) =   mnist.load_data()

Reshape the images

In [ ]:
if k.image_data_format()=='channels_first':
    x_train=x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
    x_test=x_test.reshape(x_test.shape[0],img_rows,img_cols,1)
else:
    x_train=x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
    x_test=x_test.reshape(x_test.shape[0],img_rows,img_cols,1)

input_shape=(img_rows,img_cols,1)
x_train = x_train/255.0
x_test=x_test/255.0
print('x_train shape:',x_train.shape,'\nx_test shape:',x_test.shape)

Convert class vectors to binary class matrices

In [ ]:
y_train=keras.utils.to_categorical(y_train,num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)

Design the CNN architecture

In [ ]:
from keras import layers
from keras.layers import Input
from keras.layers import Dense,Dropout,Flatten, Multiply
from keras.layers import Conv2D,MaxPool2D, Activation

In [ ]:
inputs = Input(shape=input_shape)
conv1 = Conv2D(32,kernel_size=(3,3),activation='relu')(inputs)
pool1 = MaxPool2D(pool_size=(2,2))(conv1)
conv2 = Conv2D(64,kernel_size=(3,3),activation='relu')(pool1)
pool2 = MaxPool2D(pool_size=(2,2))(conv2)
#Attention
flatten = Flatten()(pool2)
dense1 = Dense(128)(flatten)
dropout1 = Dropout(0.5)(dense1)
activation1 = Activation('relu')(dropout1)
##########
attention_conv1 = Conv2D(1, (1,1), padding='same', activation='sigmoid')(conv2)
attention_mul1 = Multiply()([conv2, attention_conv1])
##########
pool3 = MaxPool2D(pool_size=(2,2))(attention_mul1)
flatten2 = Flatten()(pool3)
dense2 = Dense(num_classes)(flatten2)
activation2 = Activation('softmax')(dense2)

model = keras.Model(inputs=inputs, outputs=activation2)

model.summary()

Compile the model

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.categorical_crossentropy,
              metrics=['accuracy']
             )
model.fit(x_train,y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test,y_test)
          )

Save the Model

In [ ]:
#model.save('AttCNNmodel.h5')
#print("model is saved")